# Симплекс-метод

In [1]:
import json
import numpy as np
from symplexmethod import symplex, main, initial, dual
from transportproblem import transport_problem
from quadraticprog import qp_problem
import scipy

ROUND_DEC_POINTS = 6 # for rounding

In [2]:
tests1 = [
    {
        'c': [1, 1, 0, 0, 0],
        'A': [[-1, 1, 1, 0, 0],
              [1, 0, 0, 1, 0],
              [0, 1, 0, 0, 1]],
        'x': [0, 0, 1, 3, 2],
        'b': [1, 3, 2],
        'B': [2, 3, 4],
    },
    {
        'c': [1, 4, 1, -1],
        'A': [[3, 1, 1, 0],
              [1, -2, 0, 1]],
        'x': [0, 0, 1, 1],
        'b': [1, 1],
        'B': [2, 3],
    },
    {
        'c': [21, 20, 0, 0, 1, 0],
        'A': [[2, 1, 1, 0, 0, 0],
              [2, 5, 0, 1, 0, 0],
              [1, 0, 0, 0, 1, 0],
              [0, 1, 0, 0, 0, 1]],
        'x': [0, 0, 14, 30, 6, 5],
        'b': [14, 30, 6, 5],
        'B': [2, 3, 4, 5],
    },
    {
        'c': [0, 0, 0, 0, -1, -1],
        'A': [[-1, 4, -1, 0, 1, 0],
              [1, 2, 0, -1, 0, 1]],
        'x': [0, 0, 0, 0, 3, 9],
        'b': [3, 9],
        'B': [4, 5],
    },
]

for test in tests1:
    for key in test:
        test[key] = np.array(test[key], dtype=('float' if key != 'B' else 'int'))

### Тестирование основной фазы симплекс-метода

Сравнение результатов вызова полного симплекс-метода, реализованного в `scipy`, принимающего параметры ЗЛП 1: `c`, `A` и `b` (каноническая форма), с результатами вызова моего алгоритма основной фазы симплекс-метода для исходных данных `c`, `A`, `x` (начальный базисный допустимый план) и `B` (вектор базисных индексов), соответствующих ЗЛП 1.

In [3]:
for i, test in enumerate(tests1):
    print(f'TEST #{i+1}:')
    print('\tSCIPY:')

    try:
        solution = symplex.scipy_solve(test['c'], test['A'], test['b'])
        x = [round(x, ROUND_DEC_POINTS) for x in solution.x]
        print(f'\t\tx = {x}, sum = {np.sum(x*test["c"])}')
    except:
        print('\t\t[ERROR]')

    print('\tCUSTOM:')
    try:
        solution = main.run(test['c'], test['A'], test['x'], test['B'])
        if solution['solved']:
            if solution['unbound']:
                print('\t\t- unbound')
            else:
                print('\t\t- bound')
                x = solution['x']
                print(f'\t\t- x = {x}, sum = {np.sum(x*test["c"])}')
        else:
            details = solution.get('details')
            print('\t\t[FAILURE]; details:')
            if details:
                for line in details:
                    print(f'\t\t\t{line}')
            else:
                print('\t\t\t(no details)')
    except Exception as e:
        print(f'\t\t[ERROR]: {e}')
    print()

TEST #1:
	SCIPY:
		x = [3.0, 2.0, 2.0, 0.0, 0.0], sum = 5.0
	CUSTOM:
		- bound
		- x = [3. 2. 2. 0. 0.], sum = 5.0

TEST #2:
	SCIPY:
		x = [0.0, 1.0, 0.0, 3.0], sum = 1.0
	CUSTOM:
		- bound
		- x = [0. 1. 0. 3.], sum = 1.0

TEST #3:
	SCIPY:
		x = [5.0, 4.0, 0.0, 0.0, 1.0, 1.0], sum = 186.0
	CUSTOM:
		- bound
		- x = [5. 4. 0. 0. 1. 1.], sum = 186.0

TEST #4:
	SCIPY:
		x = [5.132649, 2.677515, 2.57741, 1.487678, 0.0, 0.0], sum = 0.0
	CUSTOM:
		- bound
		- x = [5. 2. 0. 0. 0. 0.], sum = 0.0



### Тестирование начальной фазы симплекс-метода

In [4]:
tests2 = [
    {
        'A': [[-1, -1, -1], [-2, -2, -2]],
        'b': [-10, -20]
    },
    {
        'A': [[-1, 1, 1, 0, 0],
              [1, 0, 0, 1, 0],
              [0, 1, 0, 0, 1]],
        'b': [1, 3, 2]
    },
    {
        'A': [[2, 1, 3],
              [1, -3, 1],
              [1, 11, 3]],
        'b': [1, -3, 11]
    },
    {
        'A': [[-4, -3, -2, 1, 0, 0],
              [-3, -2, -1, 0, 1, 0],
              [-1, -1, -2, 0, 0, 1]],
        'b': [-33, -23, -12]
    }
]

for test in tests2:
    for key in test:
        test[key] = np.array(test[key], dtype='float')

In [5]:
for i, test in enumerate(tests2):
    print(f'TEST #{i+1}:')

    try:
        solution = initial.run(None, test['A'], test['b'])
        if solution['success']:
            if solution['infeasible']:
                print('\t- infeasible')
            else:
                print('\t- feasible')
                x = solution['x']
                print(f'\t- x = {x}')
                B = solution['B']
                print(f'\t- B = {B}')
                A = solution['A']
                print('\t- A:')
                for row in A:
                    print(f'\t\t{row}')
                b = solution['b']
                print(f'\t- b = {b}')
        else:
            details = solution.get('details')
            print('\t[FAILURE]; details:')
            if details:
                for line in details:
                    print(f'\t\t{line}')
            else:
                print('\t\t(no details)')
    except Exception as e:
        print(f'\t\t[ERROR]: {e}')

TEST #1:
	- feasible
	- x = [10.  0.  0.]
	- B = [0]
	- A:
		[1. 1. 1.]
	- b = [10.]
TEST #2:
	- feasible
	- x = [3. 2. 2. 0. 0.]
	- B = [1, 2, 0]
	- A:
		[-1.  1.  1.  0.  0.]
		[1. 0. 0. 1. 0.]
		[0. 1. 0. 0. 1.]
	- b = [1. 3. 2.]
TEST #3:
	- feasible
	- x = [0. 1. 0.]
	- B = [1 0]
	- A:
		[2. 1. 3.]
		[-1.  3. -1.]
	- b = [1. 3.]
TEST #4:
	- feasible
	- x = [5. 3. 2. 0. 0. 0.]
	- B = [1, 0, 2]
	- A:
		[ 4.  3.  2. -1. -0. -0.]
		[ 3.  2.  1. -0. -1. -0.]
		[ 1.  1.  2. -0. -0. -1.]
	- b = [33. 23. 12.]


### Тестирование двойственого симплекс-метода

In [6]:
tests3 = [
    {
        'c': [-4, -3, -7, 0, 0],
        'A': [[-2, -1, -4, 1, 0],
              [-2, -2, -2, 0, 1]],
        'b': [-1, -1.5],
        'B': [3, 4]
    },
    # the following tests by https://github.com/UnstoppableGuy :
    {
        'c': [2, 2, 1, -10, 1, 4, -2, -3],
        'A': [[-2, -1, 1, -7, 0, 0, 0, 2],
              [4, 2, 1, 0, 1, 5, -1, -5],
              [1, 1, 0, -1, 0, 3, -1, 1]],
        'b': [-2, 4, 3],
        'B': [1, 4, 6]
    },
    {
        'c': [12, -2, -6, 20, -18, -5, -7, -20],
        'A': [[-2, -1, 1, -7, 0, 0, 0, 2],
              [-4, 2, 1, 0, 1, 5, -1, 5],
              [1, 1, 0, 1, 4, 3, 1, 1]],
        'b': [-2, 8, -2],
        'B': [1, 3, 5]
    },
    {
        'c': [10, -2, -38, 16, -9, -9, -5, -7],
        'A': [[-2, -1, 10, -7, 1, 0, 0, 2],
              [-4, 2, 3, 0, 5, 1, -1, 0],
              [1, 1, 0, 1, -4, 3, -1, 1]],
        'b': [-2, -5, 2],
        'B': [1, 7, 4]
    }
]

for test in tests3:
    for key in test:
        test[key] = np.array(test[key], dtype=('float' if key != 'B' else 'int'))

In [7]:
for i, test in enumerate(tests3):
    print(f'TEST #{i+1}:')

    try:
        solution = dual.run(test['c'], test['A'], test['b'], test['B'])
        scipy_solution = symplex.scipy_solve(test['c'], test['A'], test['b'])
        if solution['solved']:
            if solution['infeasible']:
                print('\t- infeasible')
                print(f'\t- scipy: {scipy_solution.message}')
            else:
                print('\t- feasible')
                kappa = [round (x, ROUND_DEC_POINTS) for x in solution['kappa']]
                print(f'\t- kappa = {kappa}')
                scipy_kappa = [round(x, ROUND_DEC_POINTS) for x in scipy_solution.x]
                print(f'\t- kappa = {scipy_kappa} (scipy)')
                y = solution['y']
                print(f'\t- y = {y}')
                B = solution['B']
                print(f'\t- B = {B}')
        else:
            print('\t[FAILURE]')
    except Exception as e:
        print(f'\t\t[ERROR]: {e}')

TEST #1:
	- feasible
	- kappa = [0.25, 0.5, 0.0, 0.0, 0.0]
	- kappa = [0.25, 0.5, 0.0, 0.0, 0.0] (scipy)
	- y = [1. 1.]
	- B = [1 0]
TEST #2:
	- feasible
	- kappa = [0.0, 2.666667, 0.0, 0.0, 0.333333, 0.0, 0.0, 0.333333]
	- kappa = [0.0, 2.666667, 0.0, 0.0, 0.333333, 0.0, 0.0, 0.333333] (scipy)
	- y = [0.66666667 1.         0.66666667]
	- B = [1 7 4]
TEST #3:
	- infeasible
	- scipy: The algorithm terminated successfully and determined that the problem is infeasible.
TEST #4:
	- feasible
	- kappa = [1.35, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.45]
	- kappa = [1.35, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.45] (scipy)
	- y = [-2.7  -1.55 -1.6 ]
	- B = [1 7 0]


### Тестирование решения транспортной задачи

In [8]:
tests4 = [
    {
        'a': np.array([100, 300, 300], dtype='float'),
        'b': np.array([300, 200, 200], dtype='float'),
        'C': np.array([[8, 4, 1],
                       [8, 4, 3],
                       [9, 7, 5]], dtype='float')
    },
    {
        'a': np.array([0, 0, 0], dtype='float'),
        'b': np.array([0, 0, 0], dtype='float'),
        'C': np.array([[0, 0, 0],
                       [0, 0, 0],
                       [0, 0, 0]], dtype='float')
    },
]

for test in tests4:
    for key in test:
        test[key] = np.array(test[key], dtype='float')

# extra tests by https://github.com/UnstoppableGuy
with open('transportproblem/data5.json', 'r') as rf:
    tests4_extra = json.load(rf)['tests']
    for test in tests4_extra:
        for key in test:
            test[key] = np.array(test[key], dtype='float')

In [9]:
for i, test in enumerate(tests4):
    print(f'TEST #{i+1}:')

    try:
        solved, X = transport_problem.solve(test['a'], test['b'], test['C'])
        print(f'\t- X:')
        print('\t\t', end='')
        print('\n\t\t'.join([str(line) for line in X]))
    except Exception as e:
        print(f'\t\t[ERROR]: {e}')

for i, test in enumerate(tests4_extra):
    print(f'EXTRA TEST #{i+1}:')

    try:
        solved, X = transport_problem.solve(np.array(test['a']),
                                            np.array(test['b']),
                                            np.array(test['c']))
        print(f'\t- X:')
        print('\t\t', end='')
        print('\n\t\t'.join([str(line) for line in X]))
        print(f'\t- reference X:')
        print('\t\t', end='')
        ref_X = np.array(test['result'])
        print('\n\t\t'.join([str(line) for line in ref_X]))
        if np.array_equal(X, ref_X):
            print('\t\t- identical')
        else:
            print('\t\t- not identical')
        my_sum = np.sum(test['c'] * X)
        ref_sum = np.sum(test['c'] * ref_X)
        print(f'\t\t- my sum: {my_sum}')
        print(f'\t\t- ref sum: {ref_sum}')
    except Exception as e:
        print(f'\t\t[ERROR]: {e}')

TEST #1:
	- X:
		[  0.   0. 100.]
		[  0. 200. 100.]
		[300.   0.   0.]
TEST #2:
	- X:
		[0. 0. 0.]
		[0. 0. 0.]
		[0. 0. 0.]
EXTRA TEST #1:
	- X:
		[ 0.  0.  0. 11.  0.  0.]
		[  0.  80.  57. 149.   0.  36.]
		[53.  0.  0.  0.  0.  1.]
		[ 0.  0.  0.  0.  0. 35.]
		[ 0.  0.  0.  0. 71.  7.]
	- reference X:
		[ 0.  0.  0. 11.  0.  0.]
		[  0.  80.  50. 149.   0.  43.]
		[53.  0.  0.  0.  0.  1.]
		[ 0.  0.  0.  0.  0. 35.]
		[ 0.  0.  7.  0. 71.  0.]
		- not identical
		- my sum: 1320.0
		- ref sum: 1320.0
EXTRA TEST #2:
	- X:
		[ 0.  0. 28.  0.  0.  0.]
		[14.  0. 44.  0.  0.  0.]
		[ 0. 60.  0.  0. 37.  0.]
		[36.  0.  0.  0.  0. 22.]
		[ 30.   0.   0.  25. 204.   0.]
	- reference X:
		[ 0.  0. 28.  0.  0.  0.]
		[14.  0. 44.  0.  0.  0.]
		[ 0. 60.  0.  0. 37.  0.]
		[36.  0.  0.  0.  0. 22.]
		[ 30.   0.   0.  25. 204.   0.]
		- identical
		- my sum: 657.0
		- ref sum: 657.0
EXTRA TEST #3:
	- X:
		[ 26. 193.   2.   0.  21.  71.]
		[ 0.  0.  0.  0. 41.  0.]
		[ 0. 32.  0.  0.  0.  0

### Тестирование решения задачи квадратичного программирования

In [10]:
tests5 = [
    {
        'c': [-8, -6, -4, -6],
        'x': [2, 3, 0, 0],
        'D': [[2, 1, 1, 0],
                       [1, 1, 0, 0],
                       [1, 0, 1, 0],
                       [0, 0, 0, 0]],
        'A': [[1, 0, 2, 1],
              [0, 1, -1, 2]],
        'b': [2, 3],
        'Jb': [0, 1],
        'Jb_ast': [0, 1]
    },
    # the following tests by https://github.com/UnstoppableGuy
    {
        'c': [0, -1, 0],
        'x': [0, 2, 1],
        'D': [[2, -1, 0],
              [-1, 2, -1],
              [0, -1, 2]],
        'A': [[2, 1, 0],
              [0, 1, 2]],
        'Jb': [1, 2],
        'Jb_ast': [1, 2]
    },
    {
        'c': [0, 0, -2],
        'D': [[1, 0, 0],
              [0, 1, -1],
              [0, -1, 2]],
        'A': [[0, 1, 1],
              [1, 0, 1]],
        'x': [2, 4, 0],
        'Jb': [0, 1],
        'Jb_ast': [0, 1]
    },
]

for test in tests5:
    for key in test:
        if key != 'Jb' and key != 'Jb_ast':
            test[key] = np.array(test[key], dtype='float')

In [11]:
def solve_qp_scipy(G, a, C, b, x0):
    def f(x):
        return 0.5 * x@G@x + a@x

    constraints = []
    constraints += [{
        'type': 'eq',
        'fun': lambda x, C=C, b=b, i=i: (C@x - b)[i]
    } for i in range(len(C))]
    constraints += [{
        'type': 'ineq',
        'fun': lambda x: x
    } for _ in range(len(x0))]

    result = scipy.optimize.minimize(
        f, x0=x0, method='SLSQP', constraints=constraints,
        tol=1e-10, options={'maxiter': 2000})

    return result

for i, test in enumerate(tests5):
    print(f'TEST #{i+1}:')

    try:
        x = solve_qp_scipy(test['D'], test['c'], test['A'], test['A']@test['x'], test['x']).x
        x = np.array([round(el, ROUND_DEC_POINTS) for el in x])
        print(f'\t- x = {x} (ref)')
    except Exception as e:
        print(f'ERROR IN REF ALGORITHM: {e}')

    try:
        solved, unbounded, x = qp_problem.solve(test['c'], test['x'], test['D'],
                                                test['A'], test['Jb'], test['Jb_ast'])
        print(f'\t- x = {x}')
    except Exception as e:
        print(f'\tERROR: {e}')

TEST #1:
	- x = [1.7 2.4 0.  0.3] (ref)
	- x = [1.7 2.4 0.  0.3]
TEST #2:
	- x = [0.3 1.4 1.3] (ref)
	ERROR: Singular matrix
TEST #3:
	- x = [-0.  2.  2.] (ref)
	- x = [0. 2. 2.]
